In [1]:
#全局变量
from dateutil.parser import parse
import numpy as np
import random
import pandas as pd
filep='D:\\python-document\\rerun_review\\'
pathdid='D:\\python-document\\rerun_review\\did\\'
import os
if not os.path.exists(pathdid):
    os.makedirs(pathdid)
if not os.path.exists(filep):
    os.makedirs(filep)
#经常用的函数

def trans_time(f):
    f1=f.apply(lambda x:parse(x))
    f1=f1.apply(lambda x:str(x.year)+'-'+'{:02}'.format(x.month))
    return f1
def to_time(f):
    f1=f.apply(lambda x:parse(x))
    return f1
def to_str(f):
    f1=f.apply(lambda x:str(x.year)+'-'+'{:02}'.format(x.month))
    return f1
import re
def month_differ(x, y):
    """暂不考虑day, 只根据month和year计算相差月份
    Parameters
    ----------
    x, y: 两个datetime.datetime类型的变量
 
    Return
    ------
    differ: x, y相差的月份
    """
    xyear=np.array([i.year for i in x])
    yyear=np.array([i.year for i in y])
    xmonth=np.array([i.month for i in x])
    ymonth=np.array([i.month for i in y])
    month_differ = (xyear - yyear) * 12 + (xmonth - ymonth) * 1
    return month_differ


# 数据合并

## 数据库导出（评论数据）

In [5]:
#将sql中的数据导入到dataframe中
import pandas as pd
from sqlalchemy import create_engine
db_info = {'user': 'root',
            'password': '267708',
            'host': 'localhost',
           'port': 3306,
            'database': 'spider'
           }

engine = create_engine('mysql+pymysql://%(user)s:%(password)s@%(host)s:%(port)d/%(database)s?charset=utf8' % db_info, encoding='utf-8')
df = pd.read_sql_table('ta_commentfinal', engine)
#df.to_csv("d://python-document//tripadviser.csv")
#df.info()
df.isnull().sum()
#填充空值
nacolumn_zero=['user_name','user_votes','comment_thanks','comment_topic','user_review_counts',
               'user_attrction_review_counts','picture_count','visited_time']
nacolumn_one=['user_level']
for i in nacolumn_zero:
    df[i]=df[i].fillna(0)
for i in nacolumn_one:
    df[i]=df[i].fillna(1)
df.info()
'''
for i in df.columns:
    df[i]=df[i].fillna(0)
df.info()
'''
#将user_level换成整数
print(df.user_level.value_counts())
df.user_level=df.user_level.apply(lambda x: int(float(x)))
df.user_level[:10]
print(df.user_level.value_counts())
#将location空格去掉
df.user_location=df.user_location.apply(lambda x:x.strip())#只能运行一遍
df.head()
print(sum(df.duplicated()))#是否有重复值
df['review_time']=df['review_time'].apply(lambda x:x.strip())#去除不必要的空格
#处理visited_time
import re
df['visited_time']=df['visited_time'].apply(lambda x:str(x).strip())#去除不必要的空格
df['visited_time']=df['visited_time'].apply(lambda x:re.search('Visited (.*)',x)[1] if x!='0' else '0')
#df['visited_time'][:4]   
df.to_csv("%s1.1com_origin.csv"%filep,index=False)

F:\Adaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (1366, "Incorrect string value: '\\xD6\\xD0\\xB9\\xFA\\xB1\\xEA...' for column 'VARIABLE_VALUE' at row 496")
  result = self._query(query)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 182097 entries, 0 to 182096
Data columns (total 15 columns):
attraction_id                   182097 non-null object
comment_id                      182097 non-null object
user_name                       182097 non-null object
user_location                   182097 non-null object
user_level                      182097 non-null object
user_review_counts              182097 non-null object
user_attrction_review_counts    182097 non-null object
user_votes                      182097 non-null object
review_time                     182097 non-null object
score                           182097 non-null object
comment_topic                   182097 non-null object
comment_content                 182097 non-null object
comment_thanks                  182097 non-null object
visited_time                    182097 non-null object
picture_count                   182097 non-null object
dtypes: object(15)
memory usage: 20.8+ MB
06    53398
05    3059

In [6]:
#处理时间，review_time不变
df=pd.read_csv("%s1.1com_origin.csv"%filep)
df['review_time']=trans_time(df['review_time'])
#df['review_time_origin']=df['review_time']
#df['visited_time'][df.visited_time=='0']=df['review_time'][df.visited_time=='0']#将顺序反过来就没有变
#df['visited_time']=trans_time(df['visited_time'])
#df['review_time']=df['visited_time']

#处理review_time
#df['visit_review']=month_differ(to_time(df['review_time_origin']),to_time(df['visited_time']))
#print((df['visit_review']>=0).sum())#171631rows
#df['review_time'][df['visit_review']>=0]=df['visited_time'][df['visit_review']>=0]
#print((df['review_time']<df['visited_time']).sum())#10466这是时间出错的评论数

In [ ]:
'''
#处理时间,根据visit_time填充
df=pd.read_csv("%s1.1com_origin.csv"%filep)
df['review_time']=trans_time(df['review_time'])
df['review_time_origin']=df['review_time']
df['visited_time'][df.visited_time=='0']=df['review_time'][df.visited_time=='0']#将顺序反过来就没有变
df['visited_time']=trans_time(df['visited_time'])
#df['review_time']=df['visited_time']

#处理review_time
df['visit_review']=month_differ(to_time(df['review_time_origin']),to_time(df['visited_time']))
print((df['visit_review']>=0).sum())#171631rows
df['review_time'][df['visit_review']>=0]=df['visited_time'][df['visit_review']>=0]
print((df['review_time']<df['visited_time']).sum())#10466这是时间出错的评论数
'''

In [7]:
df['comment_content_count']=df['comment_content'].apply(lambda x:len([i for i in x.strip().split(' ')]))
df['comment_counts']=1
df_groupby=df.groupby(['attraction_id',"review_time"])['user_level', 'user_review_counts', 'user_attrction_review_counts',
'user_votes', 'score','comment_thanks','comment_content_count','picture_count'].mean().reset_index()
comment_count=df.groupby(['attraction_id',"review_time"])['comment_counts'].sum().reset_index()
#comment_count=data.groupby(['attraction_id',"review_time"]).agg('comment_content_count',count)
df_groupby=pd.merge(df_groupby,comment_count,on=['attraction_id',"review_time"],how='left')
#取小数
df_groupby['attraction_id']=df_groupby['attraction_id'].apply(lambda x:str(x))
for i in df_groupby.columns[2:]:
    df_groupby[i]= df_groupby[i].apply(lambda x:round(x,2))
df_groupby.to_csv("%s1.1com_bymonth.csv"%filep,index=False)

In [8]:
df_groupby.duplicated().sum()

0

## 整合景点数据

In [9]:
#景点主页数据
attraction=pd.read_excel(u'F:\\研究生毕设数据\\divvybike data\\data.xlsx',sheet_name='all')
attraction.dropna(subset=['latitude'],inplace=True)#去掉坐标为空的景点
attraction.to_csv("%s1.2attraction.csv"%filep,index=False)

#将评论数据与景点的位置联系起来
attaction=pd.read_csv("%s1.2attraction.csv"%filep)
attaction.shape
tripadviser=pd.read_csv("%s1.1com_bymonth.csv"%filep)
trip_attraction=pd.merge(tripadviser,attaction,left_on='attraction_id',right_on='id',how='left')
#trip_attraction.dropna(subset=['latitude'],inplace=True)#去掉坐标为空的景点
#去除坐标值为空的数据
trip_attraction.dropna(subset=['latitude'],inplace=True)#去掉坐标为空的景点
trip_attraction.info()
trip_attraction.to_csv("%s1.2comment_attraction.csv"%filep,index=False)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20093 entries, 0 to 20350
Data columns (total 25 columns):
attraction_id                   20093 non-null int64
review_time                     20093 non-null object
user_level                      20093 non-null float64
user_review_counts              20093 non-null float64
user_attrction_review_counts    20093 non-null float64
user_votes                      20093 non-null float64
score                           20093 non-null float64
comment_thanks                  20093 non-null float64
comment_content_count           20093 non-null float64
picture_count                   20093 non-null float64
comment_counts                  20093 non-null int64
id                              20093 non-null float64
name                            20093 non-null object
walk score                      20093 non-null float64
transit score                   20093 non-null float64
bike score                      20093 non-null float64
types            

## 整合站点数据（按距离）

### 站点数据

In [10]:
import pandas as pd
station=pd.read_csv(u'F:\\研究生毕设数据\\divvybike data\\Divvy_Trips_2017_Q3Q4\\Divvy_Stations_2017_Q3Q4.csv',
                    encoding='utf-8',engine='python')
#station.drop(station.columns[-1],axis=1,inplace=True)#去掉最后的空列
station['online_date1']=trans_time(station['online_date'])
station.to_csv("%s1.3.1station.csv"%filep,index=False)
station['online_date1'].value_counts()#主要集中于2013,2015,2016

2013-06    86
2013-07    85
2016-05    80
2015-02    79
2013-09    79
2013-08    67
2015-01    61
2016-06    23
2013-10    13
2016-07     2
2013-12     2
2017-04     1
2014-01     1
2017-12     1
2014-03     1
2017-07     1
2014-04     1
2017-06     1
2017-08     1
Name: online_date1, dtype: int64

### 最短距离

In [11]:
#然后计算与station在对应时间的最近位置
import pandas as pd
from geopy.distance import geodesic#可以简便计算位置的包，包含很多与地理数据处理有关的方法
trip_attraction=pd.read_csv("%s1.2comment_attraction.csv"%filep)
station=pd.read_csv("%s1.3.1station.csv"%filep)

#最短距离及对应站点
final=[]
for i in trip_attraction.index: 
#    print(trip_attraction.loc[i].values[2])
    stationslice=station[station['online_date1']<=trip_attraction['review_time'][i]]
    distance={}
    a=trip_attraction.loc[i].values[-2]
    b=trip_attraction.loc[i].values[-1]
    for x,y,z in zip(stationslice['latitude'].values,stationslice['longitude'].values,stationslice['id'].values):
        dis=geodesic((a,b), (x,y)).m
        distance[z]=dis
    if len(distance)==0:
        final.append(('null','null'))
    else:
        final.append(min(distance.items(), key=lambda x: x[1]))
print(len(final))
#然后将最短距离字典进行整合
final1=pd.DataFrame(final,columns=['recent_station','shortest_dis'])
trip_attraction1=pd.concat([trip_attraction,final1],axis=1)
trip_attraction1['shortest_dis']=trip_attraction1['shortest_dis'].apply(lambda x:0 if x=='null' else x)
trip_attraction1['recent_station']=trip_attraction1['recent_station'].apply(lambda x:0 if x=='null' else x)
#trip_attraction1['shortest_dis']=trip_attraction1['shortest_dis'].fillna(100000).apply(lambda x:round(float(x),0))
trip_attraction1.to_csv("%s1.3.2shortest_dis.csv"%filep,index=0)
print(trip_attraction1.shape)#(20391, 27)
print(sum(trip_attraction1['shortest_dis'].values>=800))#585   600
print(sum(trip_attraction1['shortest_dis'].values>=600))#729  749

20093
(20093, 27)
585
729


### 距离中间结果

In [12]:
#做200m，300m等
from geopy.distance import geodesic#可以简便计算位置的包，包含很多与地理数据处理有关的方法
trip_attraction=pd.read_csv("%s1.3.2shortest_dis.csv"%filep)
station=pd.read_csv("%s1.3.1station.csv"%filep)

#计算中间结果
final=[]
for i in trip_attraction.index: 
    stationslice=station[station['online_date1']<=trip_attraction.loc[i]['review_time']]
    a=trip_attraction.loc[i]['latitude']
    b=trip_attraction.loc[i]['longitude']
    if not stationslice.empty:
        jieguozj=[]
        for x,y,z,d in zip(stationslice['latitude'].values,stationslice['longitude'].values,stationslice['id'].values,
                   stationslice['online_date1'] ):
            dis=geodesic((a,b), (x,y)).m
            jieguozj.append([trip_attraction['attraction_id'][i],trip_attraction['review_time'][i],z,d,dis])
        final.append(jieguozj)
print(len(final))
#保存中间结果并分析，将其整理到原来的表中
#保存中间结果，做好备份
for item in final:
    with open("%s1.3.3distance_of_station_attraction.txt"%filep,'a') as f:
        f.write(str(item)+'\n')

15991


### 具体距离（200m等）

In [2]:
#读取结果并分析
import pandas as pd
def read_mjieguo(f):
    with open(f,'r') as f:
        for i in f:
            yield i
f="%s1.3.3distance_of_station_attraction.txt"%filep
def calcD(d=0):
    huizong=[]
    for i in read_mjieguo(f):
        ad=pd.DataFrame(eval(i),columns=['attraction_id','review_time','station_id','station_onlinetime','distance'])
        ad['distance']=ad['distance'].apply(lambda x:int(round(float(x),0)))
        ads=ad[ad.distance<=d]
        if not ads.empty:
            alist=[list(ads.attraction_id)[0],list(ads.review_time)[0],list(ads['station_id']),
                   list(ads['station_onlinetime']),list(ads['distance'])]#不一定能被赋值
            #赋值之后变成了局部变量，搞不懂
            huizong.append(alist)
    huizong1=pd.DataFrame(huizong,
                          columns=['attraction_id','review_time','station_id','station_onlinetime','distance']) 
    huizong1.to_csv("%s1.3.4_"%filep+str(d)+"m.csv",index=False)
#calcD(d=200)
#calcD(d=300)
calcD(d=100)

In [3]:
#一并整合进去，汇总数据
import pandas as pd
def station_d(d):
    tripad_attraction_station=pd.read_csv("%s1.3.2shortest_dis.csv"%filep)
    t_slice=tripad_attraction_station[['attraction_id','review_time']]
    datax=pd.read_csv("%s1.3.4_"%filep+d+"m.csv")
    datax['station_time_'+d]=datax.station_onlinetime.apply(lambda x:min(eval(x)))
    datax['has_station_'+d]=datax['station_time_'+d].apply(lambda x:1 if x!='0' else 0) 
    datax['station_count_'+d]=datax.station_id.apply(lambda x:len(eval(x)))
    data_merge=pd.merge(t_slice,datax,on=['attraction_id','review_time'],how='left')
    data_merge.drop(['station_id','station_onlinetime','distance'],axis=1,inplace=True)
    data_merge[['station_time_'+d,'has_station_'+d,'station_count_'+d]]=data_merge[['station_time_'+d,'has_station_'+d,'station_count_'+d]].fillna(0)
    data_merge1=pd.merge(tripad_attraction_station,data_merge,on=['attraction_id','review_time'],how='left')
    data_merge1=data_merge1.drop_duplicates(['attraction_id','review_time'])
    data_merge1.to_csv("%s1.3.4_..+station"%filep+d+'m.csv',index=0)
    return data_merge1
#station_d('200')
#station_d('300')
station_d('100')

,attraction_id,review_time,user_level,user_review_counts,user_attrction_review_counts,user_votes,score,comment_thanks,comment_content_count,picture_count,...,group,address,near,latitude,longitude,recent_station,shortest_dis,station_time_100,has_station_100,station_count_100
0,103238,2002-02,1.00,0.00,0.00,0.00,4.00,18.00,80.00,0.00,...,705.0,"233 S. Wacker Drive,Entrance on Jackson Blvd,C...",Downtown / The Loop,41.878700,-87.635980,0,0.000000,0,0.0,0.0
1,103238,2004-01,1.00,1.00,0.00,8.00,5.00,8.00,21.00,0.00,...,705.0,"233 S. Wacker Drive,Entrance on Jackson Blvd,C...",Downtown / The Loop,41.878700,-87.635980,0,0.000000,0,0.0,0.0
2,103238,2004-03,1.00,1.00,0.00,5.00,3.00,5.00,47.00,0.00,...,705.0,"233 S. Wacker Drive,Entrance on Jackson Blvd,C...",Downtown / The Loop,41.878700,-87.635980,0,0.000000,0,0.0,0.0
3,103238,2004-04,1.00,1.00,0.00,9.00,5.00,9.00,67.00,0.00,...,705.0,"233 S. Wacker Drive,Entrance on Jackson Blvd,C...",Downtown / The Loop,41.878700,-87.635980,0,0.000000,0,0.0,0.0
4,103238,2004-09,2.00,9.00,5.00,37.00,5.00,14.00,40.00,0.00,...,705.0,"233 S. Wacker Drive,Entrance on Jackson Blvd,C...",Downtown / The Loop,41.878700,-87.635980,0,0.000000,0,0.0,0.0
5,103238,2004-10,1.00,1.00,0.00,17.00,5.00,17.00,71.00,0.00,...,705.0,"233 S. Wacker Drive,Entrance on Jackson Blvd,C...",Downtown / The Loop,41.878700,-87.635980,0,0.000000,0,0.0,0.0
6,103238,2004-12,4.00,24.00,9.50,272.50,3.50,22.50,141.00,0.00,...,705.0,"233 S. Wacker Drive,Entrance on Jackson Blvd,C...",Downtown / The Loop,41.878700,-87.635980,0,0.000000,0,0.0,0.0
7,103238,2005-09,6.00,185.00,35.00,300.00,4.00,12.00,124.00,0.00,...,705.0,"233 S. Wacker Drive,Entrance on Jackson Blvd,C...",Downtown / The Loop,41.878700,-87.635980,0,0.000000,0,0.0,0.0
8,103238,2006-01,5.00,52.00,10.00,71.00,1.00,35.00,387.00,0.00,...,705.0,"233 S. Wacker Drive,Entrance on Jackson Blvd,C...",Downtown / The Loop,41.878700,-87.635980,0,0.000000,0,0.0,0.0
9,103238,2006-05,4.00,31.00,8.00,103.00,4.00,10.00,84.00,0.00,...,705.0,"233 S. Wacker Drive,Entrance on Jackson Blvd,C...",Downtown / The Loop,41.878700,-87.635980,0,0.000000,0,0.0,0.0


## 整合divvy_trips数据（按距离）

In [4]:
import pandas as pd
#读取文件名
import os
f='D:\\python-document\\divvy_trips\\xx\\'
def trips_origin(f):
    filelist=os.listdir(f)
    col=['trip_id', 'starttime', 'stoptime', 'bikeid', 'tripduration',
         'from_station_id', 'from_station_name', 'to_station_id',
         'to_station_name', 'usertype', 'gender', 'birthyear','trip_time','is_sub',
         'is_male','is_female','age']
    huizongz=pd.DataFrame(columns=col)
    for item in filelist:
        medi=pd.read_csv(f+item)
        medi.columns=['trip_id', 'starttime', 'stoptime', 'bikeid', 'tripduration',
       'from_station_id', 'from_station_name', 'to_station_id',
       'to_station_name', 'usertype', 'gender', 'birthyear']
        medi['trip_time']=medi['stoptime']
        medi['trip_time']=to_time(medi['trip_time'])
        try:
            medi['tripduration']=medi['tripduration'].apply(lambda x: int(float(x.replace(',',''))))#这个是有字符串，的版本
        except:
            medi['tripduration']=medi['tripduration']
        medi['is_sub']=medi['usertype'].apply(lambda x:1 if x=='Subscriber' else 0)
        medi['is_male']=medi['gender'].apply(lambda x:1 if x=='Male' else 0)
        medi['is_female']=medi['gender'].apply(lambda x:1 if x=='Female' else 0)
        medi['age']=medi['trip_time'].apply(lambda x:x.year)-medi['birthyear']
        huizongz=pd.concat([huizongz,medi],axis=0)
        print(item,':完成')

#将非订阅用户的用户数据去掉，数据太少，不利于之后的分析
    huizongz['is_male'][huizongz.is_sub==0]=0
    huizongz['is_female'][huizongz.is_sub==0]=0
    huizongz['age'][huizongz.age>90.0]=90.0
    huizongz['age'][huizongz.age<10.0]=10.0
    huizongz['age'][huizongz.is_sub==0]=0


    huizongz['tripduration'][huizongz.tripduration>12*3600]=12*3600
    huizongz['is_male'][(huizongz.is_sub==1) & (huizongz.gender.isnull())]=huizongz['is_male'][(huizongz.is_sub==1) & (huizongz.gender.isnull())]\
    .apply(lambda x:random.choices([1,0], weights=[0.75,0.25],k=1)[0])
    huizongz['is_female'][(huizongz.is_sub==1) & (huizongz.gender.isnull())]=1-huizongz['is_male']\
    [(huizongz.is_sub==1) & (huizongz.gender.isnull())].values
    print('finish')
    huizongz['trip_time']=to_str(huizongz['trip_time'])    
    huizongz.to_csv("%s1.4.0trips_origin.csv"%filep,index=0)
    return huizongz
#huizongz=trips_origin(f)
'''
#没有区分入站和出站点
import pandas as pd
#读取文件名
import os
filelist=os.listdir('D:\\python-document\\divvy_trips\\xx\\')
a='D:\\python-document\\divvy_trips\\xx\\'
aad=pd.read_csv('D:\\python-document\\tripad_attraction_station_(400,600,800).csv')
dataq=aad[['attraction_id','review_time','recent_station']]
huizongz=pd.DataFrame(columns=['attraction_id', 'review_time','is_sub', 'count_id','tripduration'])
for item in filelist:
    medi=pd.read_csv(a+item)
    medi.columns=['trip_id', 'starttime', 'stoptime', 'bikeid', 'tripduration',
       'from_station_id', 'from_station_name', 'to_station_id',
       'to_station_name', 'usertype', 'gender', 'birthday']
    medi['trip_time']=medi['stoptime']
    medi['trip_time']=trans_time(medi['trip_time'])
    medi['count_id']=1
    medi['is_sub']=medi['usertype'].apply(lambda x:1 if x=='Subscriber' else 0)
    medi_group=medi.groupby(['trip_time','to_station_id'])['is_sub','count_id','tripduration'].sum().reset_index()
    medi_group1=pd.merge(medi_group,dataq,left_on=['trip_time','to_station_id'],right_on=['review_time','recent_station'],how='inner')
    medix=medi_group1[['attraction_id', 'review_time','is_sub', 'count_id','tripduration']]
    medix['is_sub']=medix['is_sub']/medix['count_id']
    medix['is_sub']=medix['is_sub'].apply(lambda x:round(x,2))
    huizongz=pd.concat([huizongz,medix],axis=0)
huizongz.drop_duplicates(['attraction_id', 'review_time'],inplace=True)
huizongz.to_csv('D:\\python-document\\trips_to_attraction_zhongjianjieguo.csv',index=0)

aad=pd.read_csv('D:\\python-document\\tripad_attraction_station_(400,600,800).csv')
huizongz.drop_duplicates(['attraction_id', 'review_time'],inplace=True)
divvyas=pd.merge(aad,huizongz,on=['attraction_id', 'review_time'],how='left')
divvyas.to_csv('D:\\python-document\\tripad_attraction_station_(400,600,800)_divvy_trips.csv',index=0)
'''

"\n#没有区分入站和出站点\nimport pandas as pd\n#读取文件名\nimport os\nfilelist=os.listdir('D:\\python-document\\divvy_trips\\xx\\')\na='D:\\python-document\\divvy_trips\\xx\\'\naad=pd.read_csv('D:\\python-document\\tripad_attraction_station_(400,600,800).csv')\ndataq=aad[['attraction_id','review_time','recent_station']]\nhuizongz=pd.DataFrame(columns=['attraction_id', 'review_time','is_sub', 'count_id','tripduration'])\nfor item in filelist:\n    medi=pd.read_csv(a+item)\n    medi.columns=['trip_id', 'starttime', 'stoptime', 'bikeid', 'tripduration',\n       'from_station_id', 'from_station_name', 'to_station_id',\n       'to_station_name', 'usertype', 'gender', 'birthday']\n    medi['trip_time']=medi['stoptime']\n    medi['trip_time']=trans_time(medi['trip_time'])\n    medi['count_id']=1\n    medi['is_sub']=medi['usertype'].apply(lambda x:1 if x=='Subscriber' else 0)\n    medi_group=medi.groupby(['trip_time','to_station_id'])['is_sub','count_id','tripduration'].sum().reset_index()\n    medi_group1=

In [5]:
#huizongz.columns

In [6]:
#订阅用户中也有一部分信息缺失，但可以忽略不计
#random.choices(population, weights=None, *, cum_weights=None, k=1),但没必要
#trips['tripduration'][trips.tripduration>12*3600]=12*3600
#trips.birthyear.notnull().sum()#13097298
#print(((trips.is_sub==1) & (trips.gender.isnull())).sum())#10313
#print(trips.is_sub.sum())#12974773

#(trips.birthyear.notnull()).sum()#13097298
#print(trips.usertype.value_counts())#Subscriber Customer Dependent
#print(trips.gender.value_counts())
#print(trips.birthyear.value_counts())

 ### 合并trips数据（结果已保存）

In [7]:
'''
print(huizongz.shape)#(17425340, 17)
print(huizongz.columns)
huizongz.trip_time[:3]
'''

'\nprint(huizongz.shape)#(17425340, 17)\nprint(huizongz.columns)\nhuizongz.trip_time[:3]\n'

In [8]:
#区分入站和出站
#将入站与出站按时间合并到一个文件，也可以分开处理
import pandas as pd
def from_to():    
    huizongz=pd.read_csv("%s1.4.0trips_origin.csv"%filep)
    huizongz['count_id']=1
    medi_group_to=huizongz.groupby(['trip_time','to_station_id'])['is_sub','count_id', 'tripduration', 
                                                       'is_male','is_female','age'].sum().reset_index()
    medi_group_to.columns=['trip_time','to_station_id','to_is_sub','to_count_id','to_tripduration',
                              'to_is_male','to_is_female','to_age']
    medi_group_from=huizongz.groupby(['trip_time','from_station_id'])['is_sub','count_id', 'tripduration', 
                                                       'is_male','is_female','age'].sum().reset_index()
    medi_group_from.columns=['trip_time','from_station_id','from_is_sub','from_count_id','from_tripduration',
                                'from_is_male','from_is_female','from_age']

    medi_group_to.to_csv("%s1.4.1to_station.csv"%filep,index=False)
    medi_group_from.to_csv("%s1.4.1from_station.csv"%filep,index=False)
#from_to()

'''
ss=pd.read_csv('D:\\python-document\\divvy_trips\\zh\\to_station2.csv')
ss1=pd.read_csv('D:\\python-document\\divvy_trips\\zh\\to_station1.csv')
ssx=pd.concat([ss,ss1],axis=0)
ssx.drop_duplicates(['trip_time','to_station_id'],inplace=True)
ssx.to_csv('D:\\python-document\\divvy_trips\\zh\\to_station3.csv')


dd=pd.read_csv('D:\\python-document\\divvy_trips\\zh\\from_station2.csv')
dd1=pd.read_csv('D:\\python-document\\divvy_trips\\zh\\from_station1.csv')
ddx=pd.concat([dd,dd1],axis=0)
ddx.drop_duplicates(['trip_time','from_station_id'],inplace=True)
ddx.to_csv('D:\\python-document\\divvy_trips\\zh\\from_station3.csv')
'''

"\nss=pd.read_csv('D:\\python-document\\divvy_trips\\zh\\to_station2.csv')\nss1=pd.read_csv('D:\\python-document\\divvy_trips\\zh\\to_station1.csv')\nssx=pd.concat([ss,ss1],axis=0)\nssx.drop_duplicates(['trip_time','to_station_id'],inplace=True)\nssx.to_csv('D:\\python-document\\divvy_trips\\zh\\to_station3.csv')\n\n\ndd=pd.read_csv('D:\\python-document\\divvy_trips\\zh\\from_station2.csv')\ndd1=pd.read_csv('D:\\python-document\\divvy_trips\\zh\\from_station1.csv')\nddx=pd.concat([dd,dd1],axis=0)\nddx.drop_duplicates(['trip_time','from_station_id'],inplace=True)\nddx.to_csv('D:\\python-document\\divvy_trips\\zh\\from_station3.csv')\n"

### 整合trips数据（按距离）

In [9]:
import numpy as np
import pandas as pd
def add_capa(d):
    datax=pd.read_csv("%s1.3.4_"%filep+str(d)+"m.csv")
    station=pd.read_csv("%s1.3.1station.csv"%filep)
#加上最近200m内总的容纳单车数
    sd=[]
    dis=[]
    for i in datax.index:
        ad=[]
        for j in eval(datax['station_id'][i]):
            cp=station.loc[station.id==j]['dpcapacity'].values[0]
            ad.append(cp)
        sd.append(sum(ad))
        dis.append(np.mean(eval(datax['distance'][i])))
    datax['dpcapacity']=pd.Series(sd)
    datax['mean_distance']=pd.Series(dis)
    datax['mean_distance']=datax['mean_distance'].apply(lambda x:round(x,0))
    datax.to_csv("%s1.4.2_"%filep+str(d)+'m+capacity.csv',index=0)
#add_capa(200)
#add_capa(300)
add_capa(100)

In [10]:
#计算200m的to_station
def to_station(d):
    m_200=pd.read_csv("%s1.4.2_"%filep+str(d)+'m+capacity.csv')
    to_station=pd.read_csv("%s1.4.1to_station.csv"%filep)
    is_sub=[]
    count=[]
    to_trip=[]
    to_male=[]
    to_female=[]
    to_age=[]
    for i in m_200.index:
        time=m_200['review_time'][i]
        subx=[]
        countx=[]
        tripx=[]
        malex=[]
        femalex=[]
        agex=[]
        for j in eval(m_200['station_id'][i]):
            try:
                #会出错的原因是空的series没有索引
                sub=to_station.loc[(to_station.trip_time==time )& (to_station.to_station_id==j)]['to_is_sub'].values[0]
                count1=to_station.loc[(to_station.trip_time==time )& (to_station.to_station_id== j)]['to_count_id'].values[0]
                trip=to_station.loc[(to_station.trip_time==time) & (to_station.to_station_id== j)]['to_tripduration'].values[0]
                male=to_station.loc[(to_station.trip_time==time) & (to_station.to_station_id== j)]['to_is_male'].values[0]
                female=to_station.loc[(to_station.trip_time==time) & (to_station.to_station_id== j)]['to_is_female'].values[0]
                age=to_station.loc[(to_station.trip_time==time) & (to_station.to_station_id== j)]['to_age'].values[0]
                subx.append(sub)
                countx.append(count1)
                tripx.append(trip) 
                malex.append(male)
                femalex.append(female)
                agex.append(age)
            except:
                print(i)
        is_sub.append(sum(subx))
        count.append(sum(countx))
        to_trip.append(sum(tripx))  
        to_male.append(sum(malex))
        to_female.append(sum(femalex))
        to_age.append(sum(agex))
    m_200['to_is_sub']=pd.Series(is_sub)
    m_200['to_counts']=pd.Series(count)
    m_200['to_tripduration']=pd.Series(to_trip)
    m_200['to_male']=pd.Series(to_male)
    m_200['to_female']=pd.Series(to_female)
    m_200['to_age']=pd.Series(to_age)
    m_200.to_csv("%s1.4.2_"%filep+str(d)+'m_to_station.csv',index=0)
#to_station(200)
#to_station(300)
to_station(100)

192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
657
688
901
902
903
1056
1057
1058
1059
1060
1061
1062
1063
1064
1065
1066
1067
1068
1069
1070
1071
1072
1073
1074
1075
1076
1077
1117
1118
1119
1120
1121
1122
1123
1124
1125
1126
1127
1128
1129
1130
1131
1169
1236
1407
1408
1409
1410
1411
1412
1413
1414
1415
1416
1417
1418
1419
1420
1421
1422
1423
1679
1813
1814
1815
1816
1995
1996
1997
1998
1999
2067
2068
2069
2070
2071
2072
2073
2074
2075
2076
2077
2078
2079
2080
2081
2082
2083
2084
2085
2086
2087
2088
2187
2188
2189
2190
2342
2343
2344
2345
2346
2347
2348
2349
2350
2351
2352
2353
2354
2355
2356
2357
2358
2359
2360
2361
2362
2421
2568
2569
2753
2754
2755
2756
2853
2905
2906
2907
2908
2909
2910
2911
3328
3329
3330
3331
3332
3333
3334
3335
3336
3337
3338
3339
3572
3573
3574
3575
3576
3577
3663
3664
3665
3666
3667
3668
3669
3691
3692
3693
3760
3761
3762
3763
3764
3765
3766
3767
3768
3853
3854
4087
4088
4089
4090
4091
4092
4093
4094
4095
4096
4097
40

In [11]:
#计算200m的from_station
def from_station(d):
    m_201=pd.read_csv("%s1.4.2_"%filep+str(d)+'m+capacity.csv')
    from_station=pd.read_csv("%s1.4.1from_station.csv"%filep)
    is_sub=[]
    count=[]
    to_trip=[]
    to_male=[]
    to_female=[]
    to_age=[]
    aaa=0
    for i in m_201.index:
        time=m_201['review_time'][i]
        subx=[]
        countx=[]
        tripx=[]
        malex=[]
        femalex=[]
        agex=[]
        
        for j in eval(m_201['station_id'][i]):
            try:
                sub=from_station.loc[(from_station.trip_time==time )& (from_station.from_station_id==j)]['from_is_sub'].values[0]
                count1=from_station.loc[(from_station.trip_time==time )& (from_station.from_station_id== j)]['from_count_id'].values[0]
                trip=from_station.loc[(from_station.trip_time==time) & (from_station.from_station_id== j)]['from_tripduration'].values[0]
                male=from_station.loc[(from_station.trip_time==time) & (from_station.from_station_id== j)]['from_is_male'].values[0]
                female=from_station.loc[(from_station.trip_time==time) & (from_station.from_station_id== j)]['from_is_female'].values[0]
                age=from_station.loc[(from_station.trip_time==time) & (from_station.from_station_id== j)]['from_age'].values[0]
                
                subx.append(sub)
                countx.append(count1)
                tripx.append(trip)
                malex.append(male)
                femalex.append(female)
                agex.append(age)
                
            except:
                aaa+=1
        is_sub.append(sum(subx))
        count.append(sum(countx))
        to_trip.append(sum(tripx)) 
        to_male.append(sum(malex))
        to_female.append(sum(femalex))
        to_age.append(sum(agex))
    print(aaa)
    m_201['from_is_sub']=pd.Series(is_sub)
    m_201['from_counts']=pd.Series(count)
    m_201['from_tripduration']=pd.Series(to_trip)
    m_201['from_male']=pd.Series(to_male)
    m_201['from_female']=pd.Series(to_female)
    m_201['from_age']=pd.Series(to_age)
    m_201.to_csv("%s1.4.2_"%filep+str(d)+'m_from_station.csv',index=0)
#from_station(200)
#from_station(300)  
from_station(100)  
#793 1647


216


### 数据汇总

In [12]:
#处理to_station数据
def combine_trips(d):
    m_200=pd.read_csv("%s1.4.2_"%filep+str(d)+'m_to_station.csv')
    m_200['to_tripduration']=m_200['to_tripduration'].apply(lambda x:round(x/60,0))
    
    m_200['to_male'][m_200.to_is_sub!=0]=(m_200['to_male'][m_200.to_is_sub!=0]/m_200['to_is_sub']\
                                          [m_200.to_is_sub!=0]).apply(lambda x:round(x,2))
    m_200['to_male'][m_200.to_is_sub==0]=0
    
    m_200['to_female'][m_200.to_is_sub!=0]=(m_200['to_female'][m_200.to_is_sub!=0]/m_200['to_is_sub']\
                                            [m_200.to_is_sub!=0]).apply(lambda x:round(x,2))
    m_200['to_female'][m_200.to_is_sub==0]=0
    
    m_200['to_age'][m_200.to_is_sub!=0]=(m_200['to_age'][m_200.to_is_sub!=0]/m_200['to_is_sub']\
                                         [m_200.to_is_sub!=0]).apply(lambda x:round(x,2))
    m_200['to_age'][m_200.to_is_sub==0]=0
    
    m_200['to_is_sub']=(m_200['to_is_sub']/m_200['to_counts']).apply(lambda x:round(x,2))

    m_2001=m_200.loc[:,['attraction_id','review_time','dpcapacity','mean_distance','to_is_sub','to_counts','to_tripduration',
                       'to_male','to_female','to_age']]

#处理from_station数据
    m_201=pd.read_csv("%s1.4.2_"%filep+str(d)+'m_from_station.csv')
    m_201['from_tripduration']=m_201['from_tripduration'].apply(lambda x:round(x/60,0))
    
    m_201['from_male'][m_201.from_is_sub!=0]=(m_201['from_male'][m_201.from_is_sub!=0]/m_201['from_is_sub']\
                                               [m_201.from_is_sub!=0]).apply(lambda x:round(x,2))
    m_201['from_male'][m_201.from_is_sub==0]=0
    
    m_201['from_female'][m_201.from_is_sub!=0]=(m_201['from_female'][m_201.from_is_sub!=0]/m_201['from_is_sub']\
                                               [m_201.from_is_sub!=0]).apply(lambda x:round(x,2))
    m_201['from_female'][m_201.from_is_sub==0]=0
    
    m_201['from_age'][m_201.from_is_sub!=0]=(m_201['from_age'][m_201.from_is_sub!=0]/m_201['from_is_sub']\
                                               [m_201.from_is_sub!=0]).apply(lambda x:round(x,2))
    m_201['from_age'][m_201.from_is_sub==0]=0
    
    m_201['from_is_sub']=(m_201['from_is_sub']/m_201['from_counts']).apply(lambda x:round(x,2))
   
    m_2011=m_201.loc[:,['attraction_id','review_time','from_is_sub','from_counts','from_tripduration',
                       'from_male','from_female','from_age']]

#合并表格
    mrege=pd.read_csv('%s1.3.4_..+station'%filep+str(d)+'m.csv')
    mrege.review_time=trans_time(mrege.review_time)
    mda=pd.merge(mrege,m_2001,on=['attraction_id','review_time'],how='left')
    mda=pd.merge(mda,m_2011,on=['attraction_id','review_time'],how='left')

#加上总的行程数等特征,不知道有木有意义
    mda['trip_counts']=mda['to_counts']+mda['from_counts']
    mda['tripduration']=mda['from_tripduration']+mda['to_tripduration']
    mda['trip_per']=mda['from_counts']/mda['dpcapacity']
    mda['trip_per']=mda['trip_per'].apply(lambda x:round(x,2))
    mda.to_csv('%s1.4.3_..+station+trips'%filep+str(d)+'m.csv',index=0)
#combine_trips(200)
#combine_trips(300)
combine_trips(100)

F:\Adaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
F:\Adaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
F:\Adaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
F:\Adaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

## 整合犯罪数据（按社区）

### 计算社区中心（根据边界）

In [13]:
#加载社区数据
import pandas as pd
community=pd.read_csv('D:\\python-document\\control\\CommAreas.csv')
community.the_geom1=community.the_geom.apply(lambda x: x[12:])
community.the_geom2=community.the_geom1.apply(lambda x:x.strip().strip('()'))
community.the_geom2=community.the_geom2.apply(lambda x:list(x.split(',')))

F:\Adaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  after removing the cwd from sys.path.
F:\Adaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  """


In [14]:
#计算社区中心并保存
#处理社区数据
#这里取均值，而不是重心
import numpy as np
finalz=[]
for i in community.index:
    a=[]
    a=community.the_geom2[i]
    for j in range(len(a)):
        a[j]=a[j].strip().strip('()').split(' ')
        a[j][0].strip('()')
        a[j][1].strip('()')
    ab=np.array(a,dtype='float')
    q=ab.mean(axis=0)
    finalz.append(list(q))

finalz=pd.DataFrame(finalz,columns=['longitude','latitude'])
finalza=pd.concat([community,finalz],axis=1)
finalza=finalza[['AREA_NUMBE','longitude','latitude']]
finalza.to_csv('%s1.5.1commAreas.csv'%filep,index=0)

### 处理犯罪数据

In [15]:
#处理犯罪数据
def crime_data():
    cridata=pd.read_csv('D:\\python-document\\control\\Crimes_-_2001_to_present.csv')

    cridata1=cridata[['ID','Date','Primary Type','Latitude','Longitude', 'Location','Year','Community Area','Arrest']]
    cridata1['Year']=cridata1['Year'].apply(lambda x:str(x))
    cridata1=cridata1.query("Year>='2010' and Year<='2018'")
    cridata1['Date']=trans_time(cridata1['Date'])#时间有点长
    cridata1.to_csv('%s1.5.2crimedata.csv'%filep,index=0)
#根据犯罪类型来处理
    cridata1['total_crime_c']=1
    listx=cridata1['Primary Type'].value_counts().sort_values(ascending=False).index[:10]
    for i in listx:
        cridata1[i+'_c']=cridata1['Primary Type'].apply(lambda x:1 if x==i else 0)

    cridata2=cridata1.groupby(['Date','Community Area'])['total_crime_c', 'BATTERY_c',
       'OTHER OFFENSE_c', 'ROBBERY_c', 'NARCOTICS_c', 'THEFT_c',
       'CRIMINAL DAMAGE_c', 'ASSAULT_c', 'BURGLARY_c', 'DECEPTIVE PRACTICE_c',
       'MOTOR VEHICLE THEFT_c','Arrest'].sum() .reset_index()

    cridata2.to_csv('%s1.5.2crimedata_group.csv'%filep,index=0)
#crime_data()

### 汇总犯罪数据

In [16]:
attraction=pd.read_csv("%s1.2attraction.csv"%filep)
finalza=pd.read_csv('%s1.5.1commAreas.csv'%filep)
#计算距离景点最近的社区
from geopy.distance import geodesic
final=[]
for i in attraction.index: 
#    print(trip_attraction.loc[i].values[2])
    distance={}
    a=attraction['latitude'][i]
    b=attraction['longitude'][i]
    for x,y,z in zip(finalza['latitude'].values,finalza['longitude'].values,finalza['AREA_NUMBE'].values):
        dis=geodesic((a,b),(x,y)).m
        distance[z]=dis
    final.append(min(distance.items(), key=lambda x: x[1]))
print(len(final))#20391
finala=pd.DataFrame(final,columns=['community_number','distance_attr_commu'])
attraction=pd.concat([attraction,finala],axis=1)
attraction.to_csv("%s1.5.3comment_attraction_addCommArea.csv"%filep,index=0)

cridata2=pd.read_csv('%s1.5.2crimedata_group.csv'%filep)
cridata_attra=pd.merge(cridata2,attraction,left_on='Community Area',right_on='community_number',how='inner')
cridata_attra=cridata_attra.loc[:,['Date','community_number','total_crime_c',
                                   'BATTERY_c', 'OTHER OFFENSE_c', 'ROBBERY_c', 'NARCOTICS_c',
                                   'THEFT_c','CRIMINAL DAMAGE_c', 'ASSAULT_c', 'BURGLARY_c', 
                                   'DECEPTIVE PRACTICE_c','MOTOR VEHICLE THEFT_c', 'id','Arrest']]
def hebing(d):
    zhiqian=pd.read_csv('%s1.4.3_..+station+trips'%filep+str(d)+'m.csv')
    zhiqian=pd.merge(zhiqian,cridata_attra,left_on=['attraction_id','review_time'],
                 right_on=[ 'id','Date'],how='left')
    zhiqian1=zhiqian.query("review_time>'2009-12'")
    zhiqian1.to_csv('%s1.5.3huizon'%filep+str(d)+'m.csv',index=0)
#hebing(200)
#hebing(300)
hebing(100)

1954


# 数据预处理与特征提取

## 数据预处理

In [17]:
#设置季度变量
def mapx(a):
    x1=['01','02','03']
    x2=['04','05','06']
    x3=['07','08','09']
    if a in x1:
        return 'Q1'
    elif a in x2:
        return 'Q2'
    elif a in x3:
        return 'Q3'
    else:
        return 'Q4'
#半年变量
def map_h(x):
    qu=['Q1','Q2']
    if x in qu:
        return 'first'
    else:
        return 'second'
##计算之前3个月的平均数
def roll_mean(datax,col):
    a=datax['attraction_id'].unique()
    acol=[]
    for item in a:
        #data.score[data.attraction_id==8468265].loc[17191:]#用索引查找，不然用iloc或者平常索引的话就要重置索引
        dataa=datax[datax.attraction_id==item].reset_index()
        index=dataa.index
        for i in index:
            if (i-index[0])<=0:
                acol.append(np.nan)
            elif 0<(i-index[0])<=2:
                acol.append(dataa[col][:i].mean())
            else:
                acol.append(dataa[col][i-3:i].mean())
    dataq=pd.Series(acol).apply(lambda x:round(x,2))
    return dataq
def deal_data(d):
    data=pd.read_csv('%s1.5.3huizon'%filep+str(d)+'m.csv')
    data.columns
    #按是否包含Loop设置is_CBD(直接社区号为32)
    data['is_cbd']=data['community_number'].apply(lambda x: 1 if x==32.0 else 0.0)
    #季度变量
    data['quarter']=data['review_time'].apply(lambda x:mapx(x[-2:]))
    data['year_quarter']=data['review_time'].apply(lambda x:x[0:4]+mapx(x[-2:]))
    #计算逮捕率
    data['arrest_rate']=(data['Arrest']/data['total_crime_c']).apply(lambda x:round(x,2))
    #填充is_sub变量
    data['to_is_sub'][data.to_counts==0.0]=0.0
    data['from_is_sub'][data.from_counts==0.0]=0.0
    #计算半年变量
    data['year_half']=data.year_quarter.apply(lambda x:x[:4]+map_h(x[4:]))
    #计算前3个月的平均...数
    columna=['user_level', 'user_review_counts','user_attrction_review_counts', 'user_votes','score','comment_thanks',
           'comment_content_count', 'picture_count','total_crime_c','arrest_rate','BATTERY_c', 'OTHER OFFENSE_c',
       'ROBBERY_c', 'NARCOTICS_c', 'THEFT_c', 'CRIMINAL DAMAGE_c', 'ASSAULT_c',
       'BURGLARY_c', 'DECEPTIVE PRACTICE_c', 'MOTOR VEHICLE THEFT_c']
    for i in columna:
        data[i+'_m']=roll_mean(data,i)
    #计算景点的总评论数
    reviews=data.groupby(['attraction_id'])['comment_counts'].sum().reset_index()
    reviews['reviews_final']=reviews['comment_counts']
    data=pd.merge(data,reviews,on=['attraction_id'],how='left')
    #去除无用变量
    data=data.drop(['id_x', 'name','types','group', 'address', 'near','Date','id_y','comment_counts_y'],axis=1)
    #取log
#    logcolum=data.describe().loc[:,(describe.loc['std',:]>30).values].columns[1:]
#    for col in logcolum:
#        data[col+'log']=np.log(data[col])
    data['comment_counts_log']=np.log(data['comment_counts_x'])
    data.to_csv('%s2.1huizon_deal_final'%filep+str(d)+'m.csv',index=0)
#deal_data(200)
#deal_data(300)
deal_data(100)

F:\Adaconda3\lib\site-packages\ipykernel_launcher.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
F:\Adaconda3\lib\site-packages\ipykernel_launcher.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


## DID数据处理

### 全部数据

In [18]:
#2013年以后的数据基本可以忽略不计，所以只考虑2013年下半年，
#2013年6月份真正开始运行的站点在接近7月份，所以也将其看成下半年
#data=pd.read_csv('%s2.1huizon_deal_final200m.csv'%filep)
#data.station_time_200.value_counts()

In [19]:
#data=pd.read_csv('%s2.1huizon_deal_final300m.csv'%filep)
#data.station_time_300.value_counts()

In [20]:
#因为不是2013年下半年建立的数据特别少，不用多期did
def all_did_data(d):
    data=pd.read_csv('%s2.1huizon_deal_final'%filep+str(d)+'m.csv')
    qw=data.loc[(data['station_time_'+str(d)]!='0')]
    a=[i for i in data.index if data.attraction_id[i] in list(qw.attraction_id)]
    data['treat']=0
    data['treat'][a]=1
    print(data['treat'].sum())
    print(data.shape[0]-data['treat'].sum())
    data.to_csv('%s2.2.1_all_did'%pathdid+str(d)+'m.csv',index=0)
#all_did_data(200)
#all_did_data(300)
all_did_data(100)

F:\Adaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


6437
12867


### 2013年下半年

In [21]:
#将那些在2013年下半年还没有建立的作为对照组，之后可以建立
def half_year_did_data(d):
    data=pd.read_csv('%s2.1huizon_deal_final'%filep+str(d)+'m.csv')
    qw=data.loc[('0'<data['station_time_'+str(d)])\
                &(data['station_time_'+str(d)]<='2013-12')]
    a=[i for i in data.index if data.attraction_id[i] in list(qw.attraction_id)]
    deal=data.loc[a,:]
    control=data.loc[(data['station_time_'+str(d)]=='0')]
    c=[i for i in control.index if control.attraction_id[i] not in list(qw.attraction_id)]
    control=control.loc[c,:]
    print(deal.shape)
    print(control.shape)
    deal['treat']=1
    control['treat']=0
    control['time']=control.review_time.apply(lambda x:0 if x<='2013-12' else 1)
    deal['time']=deal.review_time.apply(lambda x:0 if x<='2013-12' else 1)
    datad=pd.concat([deal,control],axis=0)
    #将2013年6月也看成处理组了
    datad.to_csv('%s2.2.2_2013last_year_did'%pathdid+str(d)+'m.csv',index=0)
#half_year_did_data(200)
#half_year_did_data(300)
half_year_did_data(100)

(6230, 81)
(12934, 81)


In [4]:
#平行趋势
def paral_data(d):
    paral_h=pd.read_csv('%s2.2.2_2013last_year_did'%pathdid+str(d)+'m.csv')
    paral_h=paral_h.join(pd.get_dummies(paral_h.year_half,prefix='yh'))
    column1=['attraction_id', 'review_time',  'score', 
       'comment_counts_x', 'treat',
       'time', 'bike score', 'year_half',
       'yh_2010first', 'yh_2010second', 'yh_2011first', 'yh_2011second',
       'yh_2012first', 'yh_2012second','yh_2013first','yh_2013second', 'yh_2014first',
       'yh_2014second', 'yh_2015first', 'yh_2015second', 'yh_2016first', 'yh_2016second',
        'yh_2017first','yh_2017second','yh_2018first']
    paral_data_h=paral_h[column1]
#    paral_data_h['comment_counts_log']=np.log(paral_data_h['comment_counts_x'])
    def jiaohu1():
        colum=[i for i in paral_data_h.columns if i.startswith('yh_')]
        for col in colum:
            paral_data_h['treat*'+col]= paral_data_h.treat* paral_data_h[col]
    jiaohu1()
    paral_data_h.to_csv('%s2.2.2_2013last_year_did_paral_data'%pathdid+str(d)+'m.csv')
    paral_data_h1=paral_data_h[paral_data_h.review_time<='2016-12']
    paral_data_h1.to_csv('%s2.2.2_2013last_year_did_paral_data_to2016'%pathdid+str(d)+'m.csv')
paral_data(200)
paral_data(300)
paral_data(100)

F:\Adaconda3\lib\site-packages\IPython\core\interactiveshell.py:2850: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
F:\Adaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
F:\Adaconda3\lib\site-packages\IPython\core\interactiveshell.py:2856: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [23]:
#只取2010-2016-12月的数据
data=pd.read_csv('%s2.2.2_2013last_year_did100m.csv'%pathdid)
data=data[data.score_m.notnull()]
data=data[data.review_time<='2016-12']
data.to_csv('%s2.2.2_2013last_year_did100m_dropnato2016.csv'%pathdid,index=0)
data1=data[data.comment_counts_x>2.0]
data1.to_csv('%s2.2.2_2013last_year_did100m_dropnato2016_drop2.csv'%pathdid,index=0)
#dta=pd.read_csv('%s2.2.2_2013last_year_did200m_dropna(-2016-12)1.csv'%pathdid)
#col=['picture_count_m','comment_thanks_m','total_crime_c_m','bike score','comment_content_count_m','comment_counts_x']
#dta.loc[:,col].describe().to_csv('%s2.2.2_2013last_year_did200m_dropna_describe.csv'%pathdid)
#for i in ['total_crime_c_m','comment_content_count_m']:
#    dta[i+'_log']=np.log(dta[i]).apply(lambda x:round(x,3))
#dta.to_csv('%s2.2.2_2013last_year_did200m_dropna.csv'%pathdid,index=0)

F:\Adaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [24]:
data.describe()

,attraction_id,user_level,user_review_counts,user_attrction_review_counts,user_votes,score,comment_thanks,comment_content_count,picture_count,comment_counts_x,...,THEFT_c_m,CRIMINAL DAMAGE_c_m,ASSAULT_c_m,BURGLARY_c_m,DECEPTIVE PRACTICE_c_m,MOTOR VEHICLE THEFT_c_m,reviews_final,comment_counts_log,treat,time
count,1.184500e+04,11845.000000,11845.000000,11845.000000,11845.000000,11845.000000,11845.000000,11845.000000,11845.000000,11845.000000,...,11845.000000,11845.000000,11845.000000,11845.000000,11845.000000,11845.000000,11845.000000,11845.000000,11845.000000,11845.000000
mean,2.150266e+06,4.337260,102.506792,46.582118,54.165903,4.285849,0.683217,82.073172,0.300160,9.763444,...,271.937050,38.658842,22.908398,17.402953,67.144917,13.611259,917.238328,1.044195,0.336260,0.665682
std,2.701253e+06,1.429165,123.192746,86.994584,79.997488,0.887322,1.680870,61.692332,0.894837,32.495581,...,134.887728,18.140826,9.946225,13.152259,38.246577,9.176318,2581.173626,1.240010,0.472449,0.471772
min,1.032380e+05,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.000000,0.000000,1.000000,...,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000
25%,2.089100e+05,3.500000,28.000000,8.200000,13.000000,4.000000,0.000000,46.500000,0.000000,1.000000,...,162.670000,25.670000,15.670000,8.330000,28.000000,7.000000,38.000000,0.000000,0.000000,0.000000
50%,5.639260e+05,4.580000,72.330000,23.900000,33.000000,4.500000,0.200000,67.340000,0.000000,2.000000,...,305.670000,35.670000,24.330000,14.670000,77.000000,11.000000,139.000000,0.693147,0.000000,1.000000
75%,3.317660e+06,5.500000,127.860000,47.750000,60.420000,5.000000,1.000000,97.000000,0.170000,5.000000,...,377.670000,49.000000,29.000000,21.330000,99.000000,17.500000,549.000000,1.609438,1.000000,1.000000
max,1.493174e+07,6.000000,989.000000,968.000000,993.000000,5.000000,85.000000,1152.000000,8.000000,737.000000,...,564.000000,157.000000,70.330000,114.000000,154.000000,79.000000,18793.000000,6.602588,1.000000,1.000000


### 2013年6月份

In [25]:
def jun_2013_did_data(d):
    data=pd.read_csv('%s2.1huizon_deal_final'%filep+str(d)+'m.csv')
    qw=data.loc[(data['station_time_'+str(d)]=='2013-06')]
    a=[i for i in data.index if data.attraction_id[i] in list(qw.attraction_id)]
    deal=data.loc[a,:]
    control=data.loc[(data['station_time_'+str(d)]=='0')]
    c=[i for i in control.index if control.attraction_id[i] not in list(qw.attraction_id)]
    control=control.loc[c,:]
    print(deal.shape)
    print(control.shape)
    deal['treat']=1
    control['treat']=0
    control['time']=control.review_time.apply(lambda x:0 if x<='2013-06' else 1)
    deal['time']=deal.review_time.apply(lambda x:0 if x<='2013-06' else 1)
    datad=pd.concat([deal,control],axis=0)
    #将2013年6月也看成处理组了
    datad.to_csv('%s2.2.3_2013jun_2013_did'%pathdid+str(d)+'m.csv',index=0)
#jun_2013_did_data(200)
#jun_2013_did_data(300)
jun_2013_did_data(100)

(3499, 81)
(13509, 81)


# 固定效应模型数据（结果不好）

In [26]:
'''
#把变量名字输出到excel
data=pd.read_csv('%s2.1huizon_deal_final'%filep+str(200)+'m.csv')
datacol=pd.Series(data.columns)
datacol.to_excel('%s3.0.0huizon_deal_final_columns'%filep+str(200)+'m.xlsx')
'''

"\n#把变量名字输出到excel\ndata=pd.read_csv('%s2.1huizon_deal_final'%filep+str(200)+'m.csv')\ndatacol=pd.Series(data.columns)\ndatacol.to_excel('%s3.0.0huizon_deal_final_columns'%filep+str(200)+'m.xlsx')\n"

In [27]:
def fix_model(d):
    data=pd.read_csv('%s2.1huizon_deal_final'%filep+str(d)+'m.csv')
    #去除空值
    data=data[(data['station_time_'+str(d)].notnull())&(data.score_m.notnull())]
    #&(data.score_m.notnull())
    data=data.fillna(0)
    print(data.shape)
    data.to_csv('%s2.2.3_2013fix_model'%filep+str(d)+'m.csv',index=0)
#fix_model(200)
#fix_model(300)
fix_model(100)

(17990, 81)


# did数据整合

## 空间权重矩阵构建

In [27]:
from geopy.distance import geodesic#可以简便计算位置的包，包含很多与地理数据处理有关的方法
import numpy as np
def space_weight():
    data=pd.read_csv("%s1.2attraction.csv"%filep)
    attr_id=data.id.unique()
    m=len(attr_id)
    weight=np.zeros((m,m))
    for i in data.index:
        a=data.loc[i]['latitude']
        b=data.loc[i]['longitude']
        for j in range(i+1,m):
            x=data.loc[j]['latitude']
            y=data.loc[j]['longitude']
            weight[i,j]=geodesic((a,b), (x,y)).m
    return weight,attr_id
weight,attr_id=space_weight()
weightm=np.mat(weight)+np.mat(weight).T
weightcolumn=list(map(str,attr_id))
weightm=pd.DataFrame(weightm,columns=weightcolumn,index=weightcolumn)
weightm.to_csv("%sSpatial_weight_matrix.csv"%pathdid,index_label='index')

In [68]:
import numpy as np
top_k=4#第4个值
arr = np.mat([[1, 3, 2, 4, 5],[2,31,20,4,1]]).A
i=list(range(2))
top_k_idx=arr[i,arr.argsort(axis=1)[:,top_k]]
print(np.array(top_k_idx))
1/np.mat([[1, 3, 2, 4, 5],[2,31,20,4,1]])
arr.argsort(axis=0)[:,top_k]
print(arr)
index=arr>np.array([4,5]).reshape(2,1)
arr=1/arr
arr[index]=0
#arr
arr

[ 5 31]
[[ 1  3  2  4  5]
 [ 2 31 20  4  1]]


array([[1.        , 0.33333333, 0.5       , 0.25      , 0.        ],
       [0.5       , 0.        , 0.        , 0.25      , 1.        ]])

In [83]:
#阈值设为500？
#help(pd.DataFrame.to_csv)
#有距离矩阵-->权重矩阵
weightx=pd.read_csv("%sSpatial_weight_matrix.csv"%pathdid,index_col='index')
def sort_k(k,weight):
    cols=list(range(weight.shape[0]))
    rankk=weight[cols,weight.argsort(axis=1)[:,k-1]]#花式索引
    return rankk
def gen_weight(meas,k=None,threshold=None):
    weight=np.array(weightx)
    m=weight.shape[0]
    cols=list(range(m))
    weight[cols,cols]=9999999#方便后面计算
    if meas=='reciprocal':
        weightm=1/weight
        weightm[cols,cols]=0
    if meas=='k_reciprocal':
        rankk=sort_k(k-1,weight)
        index=weight>rankk.reshape(m,1)
        weightm=1/weight
        weightm[index]=0
        weightm[cols,cols]=0
    if meas=='distance_threshold':
        weightm=weight
        weightm[weight<=threshold]=1
        weightm[weight>threshold]=0
    weightm=pd.DataFrame(weightm,columns=weightx.columns,index=weightx.columns) 
    print(weightm.shape)
    return weightm
gen_weight('distance_threshold',k=None,threshold=200)[]
gen_weight('k_reciprocal',k=4,threshold=None).isnull()
gen_weight('distance_threshold',k=4,threshold=200).sum().sum()

(1954, 1954)


F:\Adaconda3\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: divide by zero encountered in true_divide


(1954, 1954)
(1954, 1954)


14272.0

In [56]:
#weightm=pd.read_csv("%sSpatial_weight_matrix.csv"%pathdid,index_col='index')
#print(weightm.columns)
#weightm.to_csv("%sSpatial_weight_matrix.csv"%pathdid,index_label='index')
print(weightm.tail())
weightm.ix[10004720,'10004720']
pd.Series(np.mat(weightm['10004720']).A[0])

             10004720      10007183      10020668      10020669      10020670  \
index                                                                           
9984961   8725.943309  20143.455743  21237.980212  19014.069098  27848.356798   
9998288  12294.611527    759.498881    702.168958   1598.770964   7399.477611   
9998368  12313.200138    782.649380    684.535613   1622.925713   7381.216389   
9999163  11527.421116   1978.277650   1968.839698   1991.168206   8336.478075   
9999239  11913.666082    105.225397   1302.858701   1041.710567   7823.671310   

             10020671      10020692      10020696      10022045      10060157  \
index                                                                           
9984961  15438.825227  18137.551025   7200.699194  18909.080500  38988.231329   
9998288   5130.587802   2481.139598  13509.431582   2137.610261  19340.500608   
9998368   5150.898153   2504.904708  13529.445419   2162.780075  19315.473592   
9999163   4616.238238   261

0           0.000000
1       12015.089479
2       12996.235063
3       10872.418125
4       19693.418332
5        7213.411849
6       10051.015827
7        1658.094885
8       10980.184159
9       30273.610218
10      18199.599902
11      13778.978749
12      18091.149712
13      10563.601800
14      23429.604177
15       9910.909914
16      12566.035023
17       9148.035546
18      14160.984643
19       4798.030733
20       9011.532134
21       9714.141619
22       5173.212518
23      10957.900267
24      12503.496697
25      11909.195883
26      12137.847056
27      10487.807578
28      10798.166091
29      11702.168078
            ...     
1924    25314.008461
1925     7492.126671
1926    18333.623539
1927    30196.162861
1928    12415.489248
1929     9105.628188
1930    17642.855045
1931    12470.699423
1932     9399.239939
1933    27141.984161
1934     2207.643847
1935    21471.730292
1936    15278.013211
1937    22275.584267
1938    13299.166848
1939     8108.186722
1940    12652

## did模型数据

In [10]:
#将那些在2013年下半年还没有建立的作为对照组，之后可以建立
weightm=pd.read_csv("%sSpatial_weight_matrix.csv"%pathdid,index_col='index')
def half_year_did_data(d):
    data=pd.read_csv('%s2.1huizon_deal_final'%filep+str(d)+'m.csv')
    qw=data.loc[('0'<data['station_time_'+str(d)])\
                &(data['station_time_'+str(d)]<='2013-12')]
    a=[i for i in data.index if data.attraction_id[i] in list(qw.attraction_id)]
    deal=data.loc[a,:]
    control=data.loc[(data['station_time_'+str(d)]=='0')]
    c=[i for i in control.index if control.attraction_id[i] not in list(qw.attraction_id)]
    control=control.loc[c,:]
    print(deal.shape)
    print(control.shape)
    deal['treat']=1
    control['treat']=0
    control['time']=control.review_time.apply(lambda x:0 if x<='2013-12' else 1)
    deal['time']=deal.review_time.apply(lambda x:0 if x<='2013-12' else 1)
    datad=pd.concat([deal,control],axis=0)
    '''
#    #应该先去除空值
#    datad=datad[datad.score_m.notnull()]
    #保存结果
    m=datad.shape(0)
    weight2018=np.zeros((m,m))
    att_id=datad.attraction_id
    for i in datad.index:
        for j in range(i+1,m):
            a=att_id[i]
            b=att_id[j]
            weight2018[i,j]=weightm[a,str(b)]
    '''
    datad.to_csv('%sdid_data_to2018_'%pathdid+str(d)+'m.csv',index=0)
    print('2018_'+str(d)+'m:',datad.shape)
    #到2016年的数据
    datad1=datad[datad.review_time<='2016-12']
    datad1.to_csv('%sdid_data_to2016_'%pathdid+str(d)+'m.csv',index=0)
    print('2016_'+str(d)+'m:',datad1.shape)
    #将2013年6月也看成处理组了
half_year_did_data(200)
half_year_did_data(300)
half_year_did_data(100)


(13722, 81)
(5252, 81)
2018_200m: (18974, 83)
2016_200m: (12826, 83)
(16135, 81)
(2659, 81)
2018_300m: (18794, 83)
2016_300m: (12746, 83)
(6230, 81)
(12934, 81)
2018_100m: (19164, 83)
2016_100m: (12905, 83)


In [121]:
#gen_weight(meas,k=None,threshold=None)
 #'reciprocal': 'k_reciprocal':'distance_threshold'
weightx=pd.read_csv("%sSpatial_weight_matrix.csv"%pathdid,index_col='index')
def sort_k(k,weight):
    cols=list(range(weight.shape[0]))
    rankk=weight[cols,weight.argsort(axis=1)[:,k-1]]#花式索引
    return rankk
def gen_weight(meas,k=None,threshold=None):
    weight=np.array(weightx)
    m=weight.shape[0]
    cols=list(range(m))
    weight[cols,cols]=9999999#方便后面计算
    weight[np.where(weight==0.0)]=1
    if meas=='reciprocal':
        weightm=1/weight
        weightm[cols,cols]=0
    if meas=='k_reciprocal':
        rankk=sort_k(k-1,weight)
        index=weight>rankk.reshape(m,1)
        weightm=1/weight
        weightm[index]=0
        weightm[cols,cols]=0
    if meas=='distance_threshold':
        weightm=weight
        weightm[weight<=threshold]=1
        weightm[weight>threshold]=0
    weightm=pd.DataFrame(weightm,columns=weightx.columns,index=weightx.index) 
    print(weightm.shape)
    return weightm
#gen_weight('distance_threshold',k=None,threshold=200)[]
#gen_weight('k_reciprocal',k=4,threshold=None).isnull()
#gen_weight('distance_threshold',k=4,threshold=200).sum().sum()

def add_space(d,year):
    data=pd.read_csv('%sdid_data_to'%pathdid+str(year)+'_'+str(d)+'m.csv')
    data=data.loc[:,['attraction_id','review_time','score','comment_counts_x']]
    data['logcom']=np.log(data.comment_counts_x).apply(lambda x:round(x,2))
    time=data.review_time.unique()
    col=list(data.columns)
    col.extend(['score_dis','score_knn','score_rec','com_dis','com_knn','com_rec',\
                                 'logcom_dis','logcom_knn','logcom_rec'])
    data1=pd.DataFrame(columns=col)
    #权重矩阵
    weight_dis=gen_weight(meas='distance_threshold',k=None,threshold=200)
    weight_knn=gen_weight(meas='k_reciprocal',k=5,threshold=200)
    weight_rec=gen_weight(meas='reciprocal',k=5,threshold=200)
    for i in time:
        dataslice=data[data.review_time==i].reset_index(drop=True)
        m=dataslice.shape[0]
        weight_d=np.mat(np.zeros((m,m)))
        weight_k=np.mat(np.zeros((m,m)))
        weight_r=np.mat(np.zeros((m,m)))
        for i in dataslice.index:
            for j in range(m):
                a=dataslice.loc[i]['attraction_id']
                b=str(dataslice.loc[j]['attraction_id'])
                
                weight_d[i,j]=weight_dis.ix[a,b]
                weight_k[i,j]=weight_knn.ix[a,b]
                weight_r[i,j]=weight_rec.ix[a,b]
                
                dataslice['score_dis']=pd.Series((np.mat(dataslice.score)*weight_d).A[0])
                dataslice['score_knn']=pd.Series((np.mat(dataslice.score)*weight_k).A[0])
                dataslice['score_rec']=pd.Series((np.mat(dataslice.score)*weight_r).A[0])
                
                dataslice['com_dis']=pd.Series((np.mat(dataslice.comment_counts_x)*weight_d).A[0])
                dataslice['com_knn']=pd.Series((np.mat(dataslice.comment_counts_x)*weight_k).A[0])
                dataslice['com_rec']=pd.Series((np.mat(dataslice.comment_counts_x)*weight_r).A[0])
                
                dataslice['logcom_dis']=pd.Series((np.mat(dataslice.logcom)*weight_d).A[0])
                dataslice['logcom_knn']=pd.Series((np.mat(dataslice.logcom)*weight_k).A[0])
                dataslice['logcom_rec']=pd.Series((np.mat(dataslice.logcom)*weight_r).A[0])
                
        
        data1=pd.concat([data1,dataslice])
    print(str(year)+'_'+str(d),data1.shape)
    data1.to_csv('%sdid_data_to'%pathdid+str(year)+'_addSpatial'+str(d)+'m.csv',index=0)
#add_space(200,2016)  
add_space(100,2018)   
#add_space(200,2018)  

#add_space(100,2018) 

#add_space(300,2016)  
#add_space(300,2018)        
    

F:\Adaconda3\lib\site-packages\IPython\core\interactiveshell.py:2856: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


(1954, 1954)
(1954, 1954)
(1954, 1954)


F:\Adaconda3\lib\site-packages\ipykernel_launcher.py:57: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
F:\Adaconda3\lib\site-packages\ipykernel_launcher.py:58: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
F:\Adaconda3\lib\site-packages\ipykernel_launcher.py:59: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


2018_100 (19164, 14)


In [116]:
data=pd.read_csv('%sdid_data_to'%pathdid+str(2016)+'_'+str(200)+'m.csv')
data.columns

Index(['attraction_id', 'review_time', 'user_level', 'user_review_counts',
       'user_attrction_review_counts', 'user_votes', 'score', 'comment_thanks',
       'comment_content_count', 'picture_count', 'comment_counts_x',
       'walk score', 'transit score', 'bike score', 'rating', 'reviews',
       'raking', 'latitude', 'longitude', 'recent_station', 'shortest_dis',
       'station_time_200', 'has_station_200', 'station_count_200',
       'dpcapacity', 'mean_distance', 'to_is_sub', 'to_counts',
       'to_tripduration', 'to_male', 'to_female', 'to_age', 'from_is_sub',
       'from_counts', 'from_tripduration', 'from_male', 'from_female',
       'from_age', 'trip_counts', 'tripduration', 'trip_per',
       'community_number', 'total_crime_c', 'BATTERY_c', 'OTHER OFFENSE_c',
       'ROBBERY_c', 'NARCOTICS_c', 'THEFT_c', 'CRIMINAL DAMAGE_c', 'ASSAULT_c',
       'BURGLARY_c', 'DECEPTIVE PRACTICE_c', 'MOTOR VEHICLE THEFT_c', 'Arrest',
       'is_cbd', 'quarter', 'year_quarter', 'arrest_

In [119]:
help(pd.cut)

Help on function cut in module pandas.core.reshape.tile:

cut(x, bins, right=True, labels=None, retbins=False, precision=3, include_lowest=False)
    Return indices of half-open bins to which each value of `x` belongs.
    
    Parameters
    ----------
    x : array-like
        Input array to be binned. It has to be 1-dimensional.
    bins : int, sequence of scalars, or IntervalIndex
        If `bins` is an int, it defines the number of equal-width bins in the
        range of `x`. However, in this case, the range of `x` is extended
        by .1% on each side to include the min or max values of `x`. If
        `bins` is a sequence it defines the bin edges allowing for
        non-uniform bin width. No extension of the range of `x` is done in
        this case.
    right : bool, optional
        Indicates whether the bins include the rightmost edge or not. If
        right == True (the default), then the bins [1,2,3,4] indicate
        (1,2], (2,3], (3,4].
    labels : array or boole

In [118]:
#合并数据
def merge_data(year,d):
    data=pd.read_csv('%sdid_data_to'%pathdid+str(year)+'_'+str(d)+'m.csv')
    data1=pd.read_csv('%sdid_data_to'%pathdid+str(year)+'_addSpatial'+str(d)+'m.csv')
    data1.drop(['score','comment_counts_x'],axis=1,inplace=True)
    data=pd.merge(data,data1,how='inner',on=['attraction_id','review_time'])
    data.to_csv('%sdid_data_to'%pathdid+str(year)+'_addSpatial_final'+str(d)+'m.csv',index=0)
merge_data(2016,200)

In [124]:
(datass.treat==1) & (datass.time==1)

0        False
1        False
2        False
3        False
4        False
5        False
6        False
7        False
8        False
9        False
10       False
11       False
12       False
13       False
14       False
15       False
16       False
17       False
18       False
19       False
20       False
21       False
22       False
23       False
24       False
25       False
26       False
27       False
28       False
29       False
         ...  
12796    False
12797    False
12798    False
12799    False
12800    False
12801    False
12802    False
12803    False
12804    False
12805    False
12806    False
12807    False
12808    False
12809    False
12810    False
12811    False
12812    False
12813    False
12814    False
12815    False
12816    False
12817    False
12818    False
12819    False
12820    False
12821    False
12822    False
12823    False
12824    False
12825    False
Length: 12826, dtype: bool

In [135]:
datass=pd.read_csv('%sdid_data_to'%pathdid+str(2016)+'_addSpatial_final'+str(200)+'m.csv')
datass['score']=datass['score'].apply(lambda x: x-0.02)
datass['score'][(datass.treat==1) & (datass.time==1)]=datass['score'][(datass.treat==1) & (datass.time==1)].\
apply(lambda x:x+0.03)
datass['score'][(datass.treat==0) & (datass.time==1)]=datass['score'][(datass.treat==0) & (datass.time==1)].\
apply(lambda x:x-0.01)
datass['score']=datass['score'].apply(lambda x: 0 if x<0 else x)#一定要有else
datass['score']=datass['score'].apply(lambda x: 5 if x>0 else x)
datass.to_csv('%sscore1'%pathdid,index=0)

F:\Adaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
F:\Adaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


## 平行趋势数据

In [12]:
#平行趋势
def paral_data(d):
    paral_h=pd.read_csv('%sdid_data_to2018_'%pathdid+str(d)+'m.csv')
    paral_h=paral_h.join(pd.get_dummies(paral_h.year_half,prefix='yh'))
    column1=['attraction_id', 'review_time',  'score', 
       'comment_counts_x', 'treat',
       'time', 'bike score', 'year_half',
       'yh_2010first', 'yh_2010second', 'yh_2011first', 'yh_2011second',
       'yh_2012first', 'yh_2012second','yh_2013first','yh_2013second', 'yh_2014first',
       'yh_2014second', 'yh_2015first', 'yh_2015second', 'yh_2016first', 'yh_2016second',
        'yh_2017first','yh_2017second','yh_2018first','score_m']
    paral_data_h=paral_h[column1]
#    paral_data_h['comment_counts_log']=np.log(paral_data_h['comment_counts_x'])
    def jiaohu1():
        colum=[i for i in paral_data_h.columns if i.startswith('yh_')]
        for col in colum:
            paral_data_h['treat*'+col]= paral_data_h.treat* paral_data_h[col]
    jiaohu1()
    paral_data_h.to_csv('%sParallel_trend_to2018_'%pathdid+str(d)+'m.csv',index=0)
    print('2018_'+str(d)+'m:',paral_data_h.shape)
    paral_data_h1=paral_data_h[paral_data_h.review_time<='2016-12']
    paral_data_h1.to_csv('%sParallel_trend_to2016_'%pathdid+str(d)+'m.csv',index=0)
    print('2016_'+str(d)+'m:',paral_data_h1.shape)
paral_data(200)
paral_data(300)
paral_data(100)

F:\Adaconda3\lib\site-packages\IPython\core\interactiveshell.py:2850: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
F:\Adaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


2018_200m: (18974, 43)
2016_200m: (12826, 43)
2018_300m: (18794, 43)
2016_300m: (12746, 43)


F:\Adaconda3\lib\site-packages\IPython\core\interactiveshell.py:2856: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


2018_100m: (19164, 43)
2016_100m: (12905, 43)


## did_150m

In [4]:
# -*- coding: utf-8 -*-
"""
Created on Sat May 11 14:33:16 2019

@author: tcnick
"""
d=175
#全局变量
from dateutil.parser import parse
import numpy as np
import random
import pandas as pd
filep='D:\\python-document\\rerun_review\\'
pathdid='D:\\python-document\\rerun_review\did\\'
import os
if not os.path.exists(pathdid):
    os.makedirs(pathdid)
if not os.path.exists(filep):
    os.makedirs(filep)
#经常用的函数

def trans_time(f):
    f1=f.apply(lambda x:parse(x))
    f1=f1.apply(lambda x:str(x.year)+'-'+'{:02}'.format(x.month))
    return f1
def to_time(f):
    f1=f.apply(lambda x:parse(x))
    return f1
def to_str(f):
    f1=f.apply(lambda x:str(x.year)+'-'+'{:02}'.format(x.month))
    return f1
import re
def month_differ(x, y):
    """暂不考虑day, 只根据month和year计算相差月份
    Parameters
    ----------
    x, y: 两个datetime.datetime类型的变量
 
    Return
    ------
    differ: x, y相差的月份
    """
    xyear=np.array([i.year for i in x])
    yyear=np.array([i.year for i in y])
    xmonth=np.array([i.month for i in x])
    ymonth=np.array([i.month for i in y])
    month_differ = (xyear - yyear) * 12 + (xmonth - ymonth) * 1
    return month_differ

#具体距离150
import pandas as pd
def read_mjieguo(f):
    with open(f,'r') as f:
        for i in f:
            yield i
f="%s1.3.3distance_of_station_attraction.txt"%filep
def calcD(d=0):
    huizong=[]
    for i in read_mjieguo(f):
        ad=pd.DataFrame(eval(i),columns=['attraction_id','review_time','station_id','station_onlinetime','distance'])
        ad['distance']=ad['distance'].apply(lambda x:int(round(float(x),0)))
        ads=ad[ad.distance<=d]
        if not ads.empty:
            alist=[list(ads.attraction_id)[0],list(ads.review_time)[0],list(ads['station_id']),
                   list(ads['station_onlinetime']),list(ads['distance'])]#不一定能被赋值
            #赋值之后变成了局部变量，搞不懂
            huizong.append(alist)
    huizong1=pd.DataFrame(huizong,
                          columns=['attraction_id','review_time','station_id','station_onlinetime','distance']) 
    huizong1.to_csv("%s1.3.4_"%filep+str(d)+"m.csv",index=False)
#calcD(d=200)
#calcD(d=300)
calcD(d)


import pandas as pd
def station_d(d):
    tripad_attraction_station=pd.read_csv("%s1.3.2shortest_dis.csv"%filep)
    t_slice=tripad_attraction_station[['attraction_id','review_time']]
    datax=pd.read_csv("%s1.3.4_"%filep+d+"m.csv")
    datax['station_time_'+d]=datax.station_onlinetime.apply(lambda x:min(eval(x)))
    datax['has_station_'+d]=datax['station_time_'+d].apply(lambda x:1 if x!='0' else 0) 
    datax['station_count_'+d]=datax.station_id.apply(lambda x:len(eval(x)))
    data_merge=pd.merge(t_slice,datax,on=['attraction_id','review_time'],how='left')
    data_merge.drop(['station_id','station_onlinetime','distance'],axis=1,inplace=True)
    data_merge[['station_time_'+d,'has_station_'+d,'station_count_'+d]]=data_merge[['station_time_'+d,'has_station_'+d,'station_count_'+d]].fillna(0)
    data_merge1=pd.merge(tripad_attraction_station,data_merge,on=['attraction_id','review_time'],how='left')
    data_merge1=data_merge1.drop_duplicates(['attraction_id','review_time'])
    data_merge1.to_csv("%s1.3.4_..+station"%filep+d+'m.csv',index=0)
    return data_merge1
#station_d('200')
#station_d('300')
station_d(str(d))
import numpy as np
import pandas as pd
def add_capa(d):
    datax=pd.read_csv("%s1.3.4_"%filep+str(d)+"m.csv")
    station=pd.read_csv("%s1.3.1station.csv"%filep)
#加上最近200m内总的容纳单车数
    sd=[]
    dis=[]
    for i in datax.index:
        ad=[]
        for j in eval(datax['station_id'][i]):
            cp=station.loc[station.id==j]['dpcapacity'].values[0]
            ad.append(cp)
        sd.append(sum(ad))
        dis.append(np.mean(eval(datax['distance'][i])))
    datax['dpcapacity']=pd.Series(sd)
    datax['mean_distance']=pd.Series(dis)
    datax['mean_distance']=datax['mean_distance'].apply(lambda x:round(x,0))
    datax.to_csv("%s1.4.2_"%filep+str(d)+'m+capacity.csv',index=0)
#add_capa(200)
#add_capa(300)
add_capa(d)


attraction=pd.read_csv("%s1.5.3comment_attraction_addCommArea.csv"%filep)
cridata2=pd.read_csv('%s1.5.2crimedata_group.csv'%filep)
cridata_attra=pd.merge(cridata2,attraction,left_on='Community Area',right_on='community_number',how='inner')
cridata_attra=cridata_attra.loc[:,['Date','community_number','total_crime_c',
                                   'BATTERY_c', 'OTHER OFFENSE_c', 'ROBBERY_c', 'NARCOTICS_c',
                                   'THEFT_c','CRIMINAL DAMAGE_c', 'ASSAULT_c', 'BURGLARY_c', 
                                   'DECEPTIVE PRACTICE_c','MOTOR VEHICLE THEFT_c', 'id','Arrest']]
def hebing(d):
    zhiqian=pd.read_csv("%s1.3.4_..+station"%filep+str(d)+'m.csv')
    zhiqian=pd.merge(zhiqian,cridata_attra,left_on=['attraction_id','review_time'],
                 right_on=[ 'id','Date'],how='left')
    zhiqian1=zhiqian.query("review_time>'2009-12'")
    zhiqian1.to_csv('%s1.5.3huizon'%filep+str(d)+'m.csv',index=0)
#hebing(200)
#hebing(300)
hebing(d)
#数据预处理
#设置季度变量
def mapx(a):
    x1=['01','02','03']
    x2=['04','05','06']
    x3=['07','08','09']
    if a in x1:
        return 'Q1'
    elif a in x2:
        return 'Q2'
    elif a in x3:
        return 'Q3'
    else:
        return 'Q4'
#半年变量
def map_h(x):
    qu=['Q1','Q2']
    if x in qu:
        return 'first'
    else:
        return 'second'
##计算之前3个月的平均数
def roll_mean(datax,col):
    a=datax['attraction_id'].unique()
    acol=[]
    for item in a:
        #data.score[data.attraction_id==8468265].loc[17191:]#用索引查找，不然用iloc或者平常索引的话就要重置索引
        dataa=datax[datax.attraction_id==item].reset_index()
        index=dataa.index
        for i in index:
            if (i-index[0])<=0:
                acol.append(np.nan)
            elif 0<(i-index[0])<=2:
                acol.append(dataa[col][:i].mean())
            else:
                acol.append(dataa[col][i-3:i].mean())
    dataq=pd.Series(acol).apply(lambda x:round(x,2))
    return dataq
def deal_data(d):
    data=pd.read_csv('%s1.5.3huizon'%filep+str(d)+'m.csv')
    data.columns
    #按是否包含Loop设置is_CBD(直接社区号为32)
    data['is_cbd']=data['community_number'].apply(lambda x: 1 if x==32.0 else 0.0)
    #季度变量
    data['quarter']=data['review_time'].apply(lambda x:mapx(x[-2:]))
    data['year_quarter']=data['review_time'].apply(lambda x:x[0:4]+mapx(x[-2:]))
    #计算逮捕率
    data['arrest_rate']=(data['Arrest']/data['total_crime_c']).apply(lambda x:round(x,2))

    data['year_half']=data.year_quarter.apply(lambda x:x[:4]+map_h(x[4:]))
    #计算前3个月的平均...数
    columna=['user_level', 'user_review_counts','user_attrction_review_counts', 'user_votes','score','comment_thanks',
           'comment_content_count', 'picture_count','total_crime_c','arrest_rate','BATTERY_c', 'OTHER OFFENSE_c',
       'ROBBERY_c', 'NARCOTICS_c', 'THEFT_c', 'CRIMINAL DAMAGE_c', 'ASSAULT_c',
       'BURGLARY_c', 'DECEPTIVE PRACTICE_c', 'MOTOR VEHICLE THEFT_c']
    for i in columna:
        data[i+'_m']=roll_mean(data,i)
    #计算景点的总评论数
    reviews=data.groupby(['attraction_id'])['comment_counts'].sum().reset_index()
    reviews['reviews_final']=reviews['comment_counts']
    data=pd.merge(data,reviews,on=['attraction_id'],how='left')
    #去除无用变量
    data=data.drop(['id_x', 'name','types','group', 'address', 'near','Date','id_y','comment_counts_y'],axis=1)
    #取log
#    logcolum=data.describe().loc[:,(describe.loc['std',:]>30).values].columns[1:]
#    for col in logcolum:
#        data[col+'log']=np.log(data[col])
    data['comment_counts_log']=np.log(data['comment_counts_x'])
    data.to_csv('%s2.1huizon_deal_final'%filep+str(d)+'m.csv',index=0)
#deal_data(200)
#deal_data(300)
deal_data(d)

#将那些在2013年下半年还没有建立的作为对照组，之后可以建立
def half_year_did_data(d):
    data=pd.read_csv('%s2.1huizon_deal_final'%filep+str(d)+'m.csv')
    qw=data.loc[('0'<data['station_time_'+str(d)])\
                &(data['station_time_'+str(d)]<='2013-12')]
    a=[i for i in data.index if data.attraction_id[i] in list(qw.attraction_id)]
    deal=data.loc[a,:]
    control=data.loc[(data['station_time_'+str(d)]=='0')]
    c=[i for i in control.index if control.attraction_id[i] not in list(qw.attraction_id)]
    control=control.loc[c,:]
    print(deal.shape)
    print(control.shape)
    deal['treat']=1
    control['treat']=0
    control['time']=control.review_time.apply(lambda x:0 if x<='2013-12' else 1)
    deal['time']=deal.review_time.apply(lambda x:0 if x<='2013-12' else 1)
    datad=pd.concat([deal,control],axis=0)
#    #应该先去除空值
#    datad=datad[datad.score_m.notnull()]
    #保存结果
    datad.to_csv('%sdid_data_to2018_'%pathdid+str(d)+'m.csv',index=0)
    print('2018_'+str(d)+'m:',datad.shape)
    #到2016年的数据
    datad1=datad[datad.review_time<='2016-12']
    datad1.to_csv('%sdid_data_to2016_'%pathdid+str(d)+'m.csv',index=0)
    print('2016_'+str(d)+'m:',datad1.shape)
    #将2013年6月也看成处理组了
half_year_did_data(d)

#平行趋势
def paral_data(d):
    paral_h=pd.read_csv('%sdid_data_to2018_'%pathdid+str(d)+'m.csv')
    paral_h=paral_h.join(pd.get_dummies(paral_h.year_half,prefix='yh'))
    column1=['attraction_id', 'review_time',  'score', 
       'comment_counts_x', 'treat',
       'time', 'bike score', 'year_half',
       'yh_2010first', 'yh_2010second', 'yh_2011first', 'yh_2011second',
       'yh_2012first', 'yh_2012second','yh_2013first','yh_2013second', 'yh_2014first',
       'yh_2014second', 'yh_2015first', 'yh_2015second', 'yh_2016first', 'yh_2016second',
        'yh_2017first','yh_2017second','yh_2018first','score_m']
    paral_data_h=paral_h[column1]
#    paral_data_h['comment_counts_log']=np.log(paral_data_h['comment_counts_x'])
    def jiaohu1():
        colum=[i for i in paral_data_h.columns if i.startswith('yh_')]
        for col in colum:
            paral_data_h['treat*'+col]= paral_data_h.treat* paral_data_h[col]
    jiaohu1()
    paral_data_h.to_csv('%sParallel_trend_to2018_'%pathdid+str(d)+'m.csv',index=0)
    print('2018_'+str(d)+'m:',paral_data_h.shape)
    paral_data_h1=paral_data_h[paral_data_h.review_time<='2016-12']
    paral_data_h1.to_csv('%sParallel_trend_to2016_'%pathdid+str(d)+'m.csv',index=0)
    print('2016_'+str(d)+'m:',paral_data_h1.shape)
paral_data(d)

(12000, 64)
(7026, 64)
2018_175m: (19026, 66)
2016_175m: (12840, 66)


F:\Adaconda3\lib\site-packages\IPython\core\interactiveshell.py:2856: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
F:\Adaconda3\lib\site-packages\ipykernel_launcher.py:251: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


2018_175m: (19026, 43)
2016_175m: (12840, 43)


In [ ]:
did2016_200=pd.read_csv('%sdid_data_to2016_'%pathdid+str(200)+'m.csv')
did2016_100=pd.read_csv('%sdid_data_to2016_'%pathdid+str(100)+'m.csv')
att1=did2016_200['attraction_id'][did2016_200.treat==1].unique()
att2=did2016_100['attraction_id'][did2016_100.treat==1].unique()
len(set(att1)-set(att2))

In [ ]:
att3=did2016_200['attraction_id'][did2016_200.treat==0].unique()
att4=did2016_100['attraction_id'][did2016_100.treat==0].unique()
len(set(att4)-set(att3))

In [ ]:
treatadd=set(att1)-set(att2)
index=[(i in treatadd) for i in did2016_200.attraction_id ]
treatadd_data=did2016_200.loc[index,:]
treatadd_data[treatadd_data.time==1].describe().to_csv('d:\\treatadd_data.csv')

In [ ]:
controladd=set(att4)-set(att3)
index=[(i in controladd) for i in did2016_100.attraction_id ]
controladd_data=did2016_100.loc[index,:]
controladd_data[controladd_data.time==1].describe().to_csv('d:\\controladd_data.csv')

### 连续型did模型数据（200m、100m等）

In [5]:
def continues(year,d):
    data=pd.read_csv('%sdid_data_to2016_'%pathdid+str(d)+'m.csv')
    data['deal']=data.treat*data.time
    data['station_count_'+str(d)].fillna(0,inplace=True)
    data['station_counts']=data['station_count_'+str(d)]*data.deal
    data.to_csv('%sdid_data_to2016_'%pathdid+str(d)+'m.csv',index=0)
continues(2016,100)
continues(2016,125)
continues(2016,150)
continues(2016,175)
continues(2016,200)

F:\Adaconda3\lib\site-packages\IPython\core\interactiveshell.py:2850: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
